#### Notes:
1. Python ASN.1 tools library does not yet support parameterization, meaning DATE-TIME-FRACTIONAL isn't accepted. Test examples without it for now.
2. Schema is compiled from file, but message to be encoded currently must be input as a python dict.

In [1]:
import numpy as np
from datetime import datetime
import asn1tools
import asn1
import pyasn1
#import asn1vnparser

## Compile schema from file

In [2]:
schema_path = '/Users/michaelbondin/ASN1-Rad-Data-Streaming-Standard/standard_schema_complete_without_date-time-fractional.asn'
encoding_rule = 'der'
spec = asn1tools.compile_files(schema_path, encoding_rule)

## Provide message as python dict

#### Example Gross Gamma Counts Radiation Data Packet

In [31]:
message = {
    'radiationDataPacketNumber'    : 3,
    'timeInformationRadiationData' : {'relativeTime'     : 3010,
                                      'accumulationTime' : 2000},
    'detectorsListRadiationData'   : [{'detectorID'  : 0,
                                       'counts'      : 6489,
                                       'derivedData' : {'countRate'    : 3244.5,
                                                        'exposure'     : 20.028,
                                                        'exposureRate' : 72100.0,
                                                        'alarm1'       : True,
                                                        'deadTime'     : 10.0,
                                                        'liveTime'     : 900.0}
                                       }],
    'endRadiationDataPacket'       : 1110000
}

message

{'radiationDataPacketNumber': 3,
 'timeInformationRadiationData': {'relativeTime': 3010,
  'accumulationTime': 2000},
 'detectorsListRadiationData': [{'detectorID': 0,
   'counts': 6489,
   'derivedData': {'countRate': 3244.5,
    'exposure': 20.028,
    'exposureRate': 72100.0,
    'alarm1': True,
    'deadTime': 10.0,
    'liveTime': 900.0}}],
 'endRadiationDataPacket': 1110000}

In [54]:
from datetime import datetime

message = {
    'instrumentStatusPacketNumber'     : 2,
    'timeInformationInstrumentStatus'  : {'relativeTime'     : 1005,
                                         'timeStamp'        : datetime.fromisoformat('2023-03-20T15:23:27')},
    'instrumentStatusInformation'      : {'instrumentTemperature' : 23,
                                         'instrumentPower'       : ('instrumentBatteryRemainingCharge', 70),
                                         'instrumentHealth'      : True},
    'detectorsListInformationStatus'   : [{'detectorID'          : 0,
                                           'detectorVoltage'     : -1290,
                                           'detectorTemperature' : 34
                                          }],
    'endInstrumentStatusPacket'       : 1111000
}

message

{'instrumentStatusPacketNumber': 2,
 'timeInformationInstrumentStatus': {'relativeTime': 1005,
  'timeStamp': datetime.datetime(2023, 3, 20, 15, 23, 27)},
 'instrumentStatusInformation': {'instrumentTemperature': 23,
  'instrumentPower': ('instrumentBatteryRemainingCharge', 70),
  'instrumentHealth': True},
 'detectorsListInformationStatus': [{'detectorID': 0,
   'detectorVoltage': -1290,
   'detectorTemperature': 34}],
 'endInstrumentStatusPacket': 1111000}

In [55]:
encoded = spec.encode('InstrumentStatus', message)

In [40]:
encoded = spec.encode('RadiationData', message)

EncodeError: RadiationData: Sequence member 'timeInformationRadiationData' not found in {'instrumentStatusPacketNumber': 2, 'timeInformationInstrumentStatus': {'relativeTime': 1005, 'timeStamp': datetime.datetime(2023, 3, 20, 15, 23, 27)}, 'instrumentStatusInformation': {'instrumentTemperature': 23, 'instrumentPower': ('instrumentBatteryRemainingCharge', 70), 'instrumentHealth': True}, 'detectorsListInformationStatus': [{'detectorID': 0, 'detectorVoltage': -1290, 'detectorTemperature': 34}], 'endInstrumentStatusPacket': 1110000}.

In [56]:
print(encoded)

bytearray(b'06\x80\x01\x02\xa1\x14\x80\x02\x03\xed\x81\x0e20230320152327\xa2\x0b\x80\x01\x17\xa1\x03\x80\x01F\x82\x01\xff\xa3\x0e0\x0c\x80\x01\x00\x84\x04\xc0\x01\x02\x85\x86\x01"')


In [57]:
bin(int.from_bytes(encoded, byteorder="big")).strip('0b')

'1100000011011010000000000000010000001010100001000101001000000000000010000000111110110110000001000011100011001000110000001100100011001100110000001100110011001000110000001100010011010100110010001100110011001000110111101000100000101110000000000000010001011110100001000000111000000000000001010001101000001000000001111111111010001100001110001100000000110010000000000000010000000010000100000001001100000000000001000000101000010110000110000000010010001'

In [58]:
print("Length of encoded message:", len(encoded), "bytes")

Length of encoded message: 56 bytes


In [59]:
bin(sum(encoded))[-8:]

'01111000'